In [1]:
import pandas as pd
import glob
import bamboolib
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

## Read files

In [2]:
def read_file_who(folder_path,type_name,page,skip_row):
    # get a list of all xlsx files in the folder
    df_all = []
    for (path,name) in zip(folder_path,type_name):
        file_list = glob.glob(path + '/*.xlsx')
        df_list = []
        # loop through each file and read the specified sheet
        for file_name in file_list:
            excel_file = pd.ExcelFile(file_name)
            if page == None:
                sheet_names = excel_file.sheet_names
            else:
                sheet_names = page
            # read the file and the specified sheet
            for sheet in sheet_names:
                if skip_row == None:
                    df = pd.read_excel(file_name, sheet_name=sheet)
                else:
                    df = pd.read_excel(file_name, sheet_name=sheet,skiprows=skip_row,header = 0)
                df['Option'] = sheet
                df_list.append(df)
                output_df = pd.concat(df_list)
        output_df['Type'] = name
        df_all.append(output_df)
    
    all_data_df = pd.concat(df_all)
    return all_data_df 




In [46]:
folder_path = ['raw/government_expenditure_on_routine_immunization',
    'raw/government_expenditure_on_vaccine_used_in_routine_immunization',
    'raw/total_expenditure_on_routine_immunization',
    'raw/total_expenditure_on_vaccine_used_in_routine_immunization']
type_name = ['gov_immunization','gov_vaccine','total_immunization','total_vaccine']

expenditure_df = read_file_who(folder_path,type_name,None,None)

coverage_df = read_file_who(['raw\coverage'],['coverage'],['Data'],None)

population_df = read_file_who(['raw\population'],['population'],['Estimates'],16)

gdp_df = read_file_who(['raw\GDP'],['current GDP'],['Data'],3)

land_df = read_file_who(['raw\land'],['land'],['SYB65_145_202209_Land'],1)

gdp_def_df = read_file_who(['raw\gdp_deflator'],['gdp_deflator'],['Data'],3)

## Data Preparation

In [47]:
#Standardize the country name
def replace_country_names(df, replacements):
    for original, replacement in replacements.items():
        df.loc[df['Country'] == original, 'Country'] = replacement
    return df

replacements = {
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Congo (The)': 'Congo',
    #'Democratic Republic of the Congo': 'Congo',
    'Czechia': 'Czech Republic',
    'Iran (Islamic Republic of)': 'Iran',
    'Lao People\'s Democratic Republic': 'Laos',
    'Lao People\'s Democratic Republic (the)': 'Laos',
    'North Macedonia': 'North Macedonia',
    'Republic of Macedonia': 'North Macedonia',
    'The former Yugoslav Republic of Macedonia': 'North Macedonia',
    'Netherlands': 'The Netherlands',
    'Saint Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Saint Vincent and The Grenadines': 'Saint Vincent and the Grenadines',
    'Sudan': 'Sudan',
    'South Sudan': 'South Sudan',
    'Syrian Arab Republic': 'Syria',
    'Syria': 'Syria',
    'Tanzania': 'Tanzania',
    'United Republic of Tanzania': 'Tanzania',
    'Turks and Caicos Islands': 'Turks and Caicos Islands',
    'Turks and Caicos Islands, The': 'Turks and Caicos Islands',
    'United Kingdom': 'United Kingdom',
    'England': 'United Kingdom',
    'Wales': 'United Kingdom',
    'Scotland': 'United Kingdom',
    'Northern Ireland': 'United Kingdom',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'Bolivia (Plurin. State of)':'Bolivia',
    'Côte d’Ivoire':"Côte d'Ivoire",
    'Dem. Rep. of the Congo':'Democratic Republic of the Congo',
    "Lao People's Dem. Rep.":'Laos',
    "Dem. People's Rep. Korea":'Democratic Peoples Republic of Korea',
    'State of Palestine' : 'West Bank and Gaza',
    'Russian Federation' :"Russia",
    'Türkiye':'Turkey',
    'United Rep. of Tanzania':'Tanzania',
    'Saint Vincent & Grenadines':'Saint Vincent and the Grenadines',
    'Venezuela (Boliv. Rep. of)' : 'Venezuela',
    'Bahamas, The': 'Bahamas',
    "Cote d'Ivoire":"Côte d'Ivoire",
    "Congo, Dem. Rep.":'Democratic Republic of the Congo',
    'Egypt, Arab Rep.': 'Egypt',
    'Micronesia, Fed. Sts.':'Micronesia',
    'Gambia, The': 'Gambia',
    'Iran, Islamic Rep.':'Iran',
    'Kyrgyz Republic':'Kyrgyzstan',
    'St. Kitts and Nevis':'Saint Kitts and Nevis',
    'Korea, Rep.':'Republic of Korea',
    'Lao PDR': 'Laos',
    'St. Lucia' : 'Saint Lucia',
    'Moldova' : 'Republic of Moldova',
    "Korea, Dem. People's Rep.":'Democratic Peoples Republic of Korea',
    'Slovak Republic': 'Slovakia',
    'Turkiye':'Turkey',
    'United States':'United States of America',
    'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Venezuela, RB':'Venezuela',
    'Yemen, Rep.': 'Yemen',
    'Syrian Arab Republic':'Syria'
}

In [48]:
def get_missing_value_percentage(expenditure_detail_df,columns_to_group=['Country'], columns_to_filter=['total_immunization in USD', 'total_vaccine in USD'], threshold=0.3,more_than = True):
    # group the data by both 'Year' and 'Country'
    grouped = expenditure_detail_df.groupby(columns_to_group)
    
    # calculate the percentage of missing values for each column within each group
    nan_percentages = grouped.apply(lambda x: x.isna().mean())
    
    # filter the results to only show the specified columns
    filtered = nan_percentages[columns_to_filter]
    
    # filter the results to only show percentages above the threshold value
    if more_than == True:
        filtered = filtered[filtered > threshold].dropna()
    else:
        filtered = filtered[filtered < threshold].dropna()
    return filtered

### GDP Deflator

In [49]:
gdp_def_df.columns = gdp_def_df.columns.astype(str)
value_vars = gdp_def_df.columns.drop(['Country Name', 'Country Code', 'Indicator Name','Indicator Code', 'Option', 'Type'])
gdp_def_melt_df = pd.melt(gdp_def_df, id_vars=['Country Name', 'Country Code', 'Indicator Name','Indicator Code', 'Option', 'Type'], 
                 value_vars=value_vars, 
                 var_name='Year', value_name='gdp deflator index')

gdp_def_melt_df['Year'] = pd.to_datetime(gdp_def_melt_df['Year'], format='%Y').dt.year
gdp_def_df = gdp_def_melt_df

gdp_def_df = gdp_def_df.rename(columns={"Country Name": 'Country'})

In [50]:
gdp_def_df = replace_country_names(gdp_def_df, replacements)
gdp_def_df['Year']= pd.to_datetime(gdp_def_df['Year'], format='%Y')

### Land

In [51]:
land_df = land_df[land_df['Series'] == 'Land area (thousand hectares)']
# Get the latest year
latest_year = land_df['Year'].max()

# Create a new dataframe with the latest values for each country/region
country_area_df = land_df.loc[land_df['Year'] == latest_year, ["Unnamed: 1", 'Value']].copy()

# Rename the 'Region/Country/Area' column to 'Country'
country_area_df = country_area_df.rename(columns={"Unnamed: 1": 'Country','Value':'land_area'})
country_area_df = replace_country_names(country_area_df, replacements)

### Goverment expenditure on immunization

In [52]:
#expenditure_df = read_file_who(folder_path,type_name,None,None)
expenditure_df.columns = expenditure_df.columns.astype(str)
value_vars = expenditure_df.columns.drop(['ISO', 'Country', 'Region', 'Gavi / Income status', 'Option', 'Type'])
expenditure_melt_df = pd.melt(expenditure_df, id_vars=['ISO', 'Country', 'Region', 'Gavi / Income status', 'Option', 'Type'], 
                 value_vars=value_vars, 
                 var_name='Year', value_name='value')

expenditure_melt_df['Year'] = pd.to_datetime(expenditure_melt_df['Year'], format='%Y').dt.year
expenditure_df = expenditure_melt_df


In [53]:
gov_type = ['gov_immunization', 'gov_vaccine']
gov_list = ['in USD', 'in USD per capita', 'in USD per surviving infant', '% government expenditure']
start = 0
for j in gov_type:
    for i in gov_list:
        if start == 0:
            expenditure_detail_df = expenditure_df[(expenditure_df['Type'] == j) & (expenditure_df['Option'] == i)][['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year', 'value']]
            expenditure_detail_df = expenditure_detail_df.rename(columns={'value': j + ' ' + i})
        else:
            df_2 = expenditure_df[(expenditure_df['Type'] == j) & (expenditure_df['Option'] == i)][['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year', 'value']]
            df_2 = df_2.rename(columns={'value': j + ' ' + i})
            expenditure_detail_df = pd.merge(expenditure_detail_df, df_2, on=['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year'], how='outer', suffixes=('', ''))
        start += 1 

total_type = ['total_immunization', 'total_vaccine']
total_list = ['in USD', 'in USD per capita', 'in USD per surviving infant']
for j in total_type:
    for i in total_list:
        df_2 = expenditure_df[(expenditure_df['Type'] == j) & (expenditure_df['Option'] == i)][['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year', 'value']]
        df_2 = df_2.rename(columns={'value': j + ' ' + i})
        expenditure_detail_df = pd.merge(expenditure_detail_df, df_2, on=['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year'], how='outer', suffixes=('', ''))

In [54]:
expenditure_detail_df = replace_country_names(expenditure_detail_df, replacements)
expenditure_detail_df['Year']= pd.to_datetime(expenditure_detail_df['Year'], format='%Y')

In [55]:
#expenditure_detail_filt_df = expenditure_detail_df[['Country','Year','Region','Gavi / Income status','total_immunization in USD per surviving infant','total_vaccine in USD per surviving infant']]
expenditure_detail_filt_df = expenditure_detail_df[['Country','Year','Region','Gavi / Income status','total_immunization in USD','gov_immunization in USD','total_vaccine in USD','total_immunization in USD per surviving infant','total_vaccine in USD per surviving infant']]

In [56]:
#expenditure_detail_filt_df = expenditure_detail_filt_df.rename(columns = {'total_immunization in USD per surviving infant':'total_immunization in USD', 'total_vaccine in USD per surviving infant':'total_vaccine in USD'})

### Immunization coverage

In [57]:
coverage_df = coverage_df.rename(columns={'NAME': 'Country','YEAR':'Year'})
coverage_df = coverage_df[coverage_df['COVERAGE_CATEGORY'].isin(['WUENIC'])]
coverage_df = coverage_df[~coverage_df['ANTIGEN'].isin(['YFV'])]
coverage_df = replace_country_names(coverage_df, replacements)
coverage_df['Year'] = pd.to_datetime(coverage_df['Year'], format='%Y')

In [58]:
# create a pivot table for each antigen
antigen_pivot = pd.pivot_table(coverage_df, index=['Country', 'Year'], 
                               columns='ANTIGEN',
                               values=['COVERAGE'], 
                               aggfunc='sum')

# flatten the column index
antigen_pivot.columns = [f'{col[1]}_{col[0]}' for col in antigen_pivot.columns]

# reset the index to make Country and Year columns
antigen_pivot = antigen_pivot.reset_index()

#rename
immunization_coverage_df = antigen_pivot

### World Population

In [59]:
#population_df = read_file_who(['raw\population'],['population'],['Estimates'],16)
population_df

cols = [i for i in range(100)]
total = population_df[cols].sum(axis=1)
cols = [i for i in range(1,6)]
total_below_5 = population_df[cols].sum(axis=1)
cols = [i for i in range(6,11)]
total_below_10 = population_df[cols].sum(axis=1)
cols = [i for i in range(10,16)]
total_below_15 = population_df[cols].sum(axis=1)
cols = [i for i in range(16,21)]
total_below_20 = population_df[cols].sum(axis=1)
cols = [i for i in range(21,26)]
total_below_25 = population_df[cols].sum(axis=1)
cols = [i for i in range(0,1)]
total_below_1 = population_df[cols].sum(axis=1)

# add the 'total' column to the DataFrame
population_df['total_population'] = total
population_df['total_population_1-5'] = total_below_5
population_df['total_population_6-10'] = total_below_10
population_df['total_population_11-15'] = total_below_15
population_df['total_population_16-20'] = total_below_20
population_df['total_population_21-25'] = total_below_25
population_df['total_population_0-1'] = total_below_1
population_df['total_population'] = population_df['total_population'] + population_df['100+']
population_df = population_df.rename(columns={'Region, subregion, country or area *': 'Country'})
population_df['Year'] = pd.to_datetime(population_df['Year'], format='%Y')
population_df = replace_country_names(population_df, replacements)
population_df.columns = [str(column) for column in population_df.columns]

In [60]:
list = ['total_population','total_population_0-1','total_population_1-5', 'total_population_6-10',
       'total_population_11-15', 'total_population_16-20']
for i in list:
    population_df[i] = pd.to_numeric(population_df[i], downcast='float', errors='coerce')

In [61]:
population_filt_df = population_df[['Country','Year','total_population','total_population_0-1','total_population_1-5','total_population_6-10',
       'total_population_11-15', 'total_population_16-20']]

### GDP

In [62]:
#gdp_df = read_file_who(['raw\GDP'],['current GDP'],['Data'],3)

gdp_df.columns = gdp_df.columns.astype(str)
value_vars = gdp_df.columns.drop(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Option', 'Type'])
gdp_melt_df = pd.melt(gdp_df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Option', 'Type'], 
                 value_vars=value_vars, 
                 var_name='Year', value_name='GDP')
gdp_df = gdp_melt_df.rename(columns={'Country Name':'Country'})
gdp_df['GDP_Million'] = gdp_df['GDP']/1000000
gdp_df['Year'] = pd.to_datetime(gdp_df['Year'], format='%Y')
gdp_df = replace_country_names(gdp_df, replacements)

#rename
gdp_filt_df = gdp_df

### Merge Data

In [97]:
merged_df = pd.merge(expenditure_detail_filt_df[['Country', 'Region', 'Gavi / Income status', 'Year',
       'total_vaccine in USD',  'total_immunization in USD', 'gov_immunization in USD','total_immunization in USD per surviving infant','total_vaccine in USD per surviving infant'
     ]].drop_duplicates(),immunization_coverage_df.drop_duplicates(), on = ['Country','Year'], how = 'left')

merged_df = pd.merge(merged_df.drop_duplicates(),gdp_filt_df[['Country','GDP_Million','Year']].drop_duplicates(), on = ['Country','Year'], how = 'left')

merged_df = pd.merge(merged_df.drop_duplicates(),population_filt_df[['Country','Year','total_population','total_population_0-1','total_population_1-5', 'total_population_6-10',
       'total_population_11-15', 'total_population_16-20']].drop_duplicates(),on = ['Country','Year'],how ='left')

merged_df = pd.merge(merged_df.drop_duplicates(),country_area_df[['Country','land_area']].drop_duplicates(),on = ['Country'],how ='left')

gdp_def_df = gdp_def_df.loc[gdp_def_df['Country'].isin(['United States of America'])]

merged_df = pd.merge(merged_df,gdp_def_df[['Year','gdp deflator index']], on = 'Year', how = 'left')
#merged_df = pd.merge(merged_df.drop_duplicates(),gdp_def_df[['Year','gdp deflator index']].drop_duplicates(), on = 'Year', how = 'left')

### Clean data

##### Deflate the data using the USD deflator index

In [98]:
cols_to_adjust = ['total_vaccine in USD', 'total_immunization in USD', 'gov_immunization in USD', 'total_immunization in USD per surviving infant', 'total_vaccine in USD per surviving infant', 'GDP_Million']

for col in cols_to_adjust:
    merged_df[f'{col}'] = (merged_df[col] / merged_df['gdp deflator index']) * 100

##### Exclude the rich country (high income)

In [99]:
merged_df = merged_df[merged_df['Gavi / Income status'] != 'High income countries']

##### Drop few vaccine due to very high missing values

In [100]:
#coverage_list = ['BCG_COVERAGE', 'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE', 'HEPB_BD_COVERAGE', 'HIB3_COVERAGE', 'MCV1_COVERAGE', 'MCV2_COVERAGE', 'PCV3_COVERAGE', 'POL3_COVERAGE', 'RCV1_COVERAGE', 'ROTAC_COVERAGE']
merged_df = merged_df.drop(['MCV2X2_COVERAGE','IPV1_COVERAGE'] , axis = 1)

##### Create new column to calculate the cost of vaccine delivery

In [101]:
merged_df['Year'] = pd.to_datetime(merged_df['Year'], format='%Y')
merged_df['total_immunization in USD'] = merged_df['total_immunization in USD'].replace(0, np.nan)
merged_df['total_vaccine in USD'] = merged_df['total_vaccine in USD'].replace(0, np.nan)
merged_df['total_vaccine_delivery_cost in USD'] = merged_df['total_immunization in USD'] -merged_df['total_vaccine in USD']


##### Change the data into Million format

In [102]:
#change the data into Million format
merged_df['Vaccine USD Mil'] = merged_df['total_vaccine in USD'] / 1000000
merged_df['Immunization USD Mil'] = merged_df['total_immunization in USD'] / 1000000
merged_df['Vaccine Delivery Cost USD Mil'] = merged_df['total_vaccine_delivery_cost in USD'] / 1000000

##### Compute the gov to total immunization ratio

In [103]:
merged_df['gov_to_total_immunization_ratio'] = merged_df['gov_immunization in USD']/merged_df['total_immunization in USD']

In [104]:
origin_merged_df = merged_df.copy()

##### Create new columns for previous year data and average previous 3 years data

In [105]:
# group the dataframe by country
grouped_df = merged_df.groupby('Country')

# Define a function to create the previous year columns in million
def create_prev_year_cols_million(col):
    return [f"{col}_prev_{i}_year_million" for i in range(1, 4)]

# Define a function to calculate the average of the previous 3 years' data considering only available years
def calculate_3_year_average(row, cols):
    return row[cols].apply(lambda x: np.nanmean(x), axis=1)

# Create new columns for 'total_vaccine in USD', 'total_immunization in USD', and 'total_vaccine_delivery_cost in USD', 
# for the value for previous year, in million
for col in ['total_vaccine in USD', 'total_immunization in USD', 'total_vaccine_delivery_cost in USD']:
    prev_year_cols = create_prev_year_cols_million(col)
    for i, prev_year_col in enumerate(prev_year_cols):
        merged_df[prev_year_col] = grouped_df[col].apply(lambda x: x.shift(periods=i+1) / 1000000)
        #print(i)
        #print(col)
        # Calculate the percentage change between the current year and the previous year
        curr_col = f"{col}_prev_1_year_million"
        pct_col = f"{col}_prev_{i+1}_year_pct_change"
        if curr_col in merged_df.columns and prev_year_col in merged_df.columns:
            merged_df[pct_col] = (merged_df[curr_col] - merged_df[prev_year_col]) / merged_df[prev_year_col]
        else:
            merged_df[pct_col] = float('nan')

    # Create a new column to store the average of the previous 3 years' data considering only available years
    avg_col = f"{col}_prev_3_year_avg_million"
    merged_df[avg_col] = calculate_3_year_average(merged_df, prev_year_cols)
    #print('mm')

In [106]:
merged_df['total_vaccine in USD_prev_1_year_pct_change'] = (merged_df['Vaccine USD Mil'] - merged_df['total_vaccine in USD_prev_1_year_million']) / merged_df['total_vaccine in USD_prev_1_year_million']
merged_df['total_immunization in USD_prev_1_year_pct_change'] = (merged_df['Immunization USD Mil'] - merged_df['total_immunization in USD_prev_1_year_million']) / merged_df['total_immunization in USD_prev_1_year_million']
merged_df['total_vaccine_delivery_cost in USD_prev_1_year_pct_change'] = (merged_df['Vaccine Delivery Cost USD Mil'] - merged_df['total_vaccine_delivery_cost in USD_prev_1_year_million']) / merged_df['total_vaccine_delivery_cost in USD_prev_1_year_million']

##### Visualize some of the data to see if there is unexpected data

In [107]:
go.Figure(
    data=[go.Histogram(x=merged_df["Vaccine Delivery Cost USD Mil"], xbins={"start": -60.0, "end": 420.0, "size": 40.0})],
    layout=go.Layout(title="Histogram of Vaccine Delivery Cost USD Mil", yaxis={"title": "Count"}, bargap=0.05),
    )

In [108]:
# Step: Keep rows where Vaccine Delivery Cost USD Mil > 0
#merged_df = merged_df.loc[merged_df['Vaccine Delivery Cost USD Mil'] > 0]

In [109]:
fig = px.histogram(merged_df.dropna(subset=['total_vaccine in USD_prev_1_year_pct_change']), x='Year', histfunc='avg', y='total_vaccine in USD_prev_1_year_pct_change')
fig

In [110]:
fig = px.histogram(merged_df.dropna(subset=['total_immunization in USD_prev_1_year_pct_change']), x='Year', histfunc='avg', y='total_immunization in USD_prev_1_year_pct_change')
fig

In [111]:
fig = px.histogram(merged_df.dropna(subset=['total_immunization in USD_prev_1_year_pct_change']), x='Year', histfunc='avg', y='total_immunization in USD_prev_1_year_pct_change')
fig

In [112]:

fig = px.histogram(merged_df, x='Year', histfunc='avg', y='total_vaccine_delivery_cost in USD_prev_1_year_pct_change')
fig

##### Exclude the immunization data that 0 or NaN

In [113]:
merged_df = merged_df.loc[merged_df['total_immunization in USD'] > 0]

In [114]:
merged_df = merged_df.loc[merged_df['gov_immunization in USD'] > 0]

In [115]:
merged_df = merged_df.sort_values(by=['total_vaccine_delivery_cost in USD_prev_1_year_pct_change'], ascending=[False])

### Feature Engineering

In [116]:
# Create a new DataFrame to store the average ratios
avg_ratios_df = pd.DataFrame(columns=['Country', 'Year', 'Avg_3yr_gov_ratio'])

# Iterate over each country in the merged_df DataFrame
for country in merged_df['Country'].unique():
    # Select only the rows for the current country
    country_df = merged_df.loc[merged_df['Country'] == country]
    
    # Iterate over each year for the current country
    for i in range(len(country_df)):
        # If the current year is null, use the latest available 3 years to calculate the average
        if pd.isnull(country_df.iloc[i]['Year']):
            latest_years = country_df['Year'].dropna().tail(3)
            if len(latest_years) < 3:
                avg_ratio = country_df['gov_to_total_immunization_ratio'].dropna().mean()
            else:
                avg_ratio = country_df.loc[country_df['Year'].isin(latest_years)].\
                            iloc[-3:]['gov_to_total_immunization_ratio'].mean()
        # Otherwise, calculate the average of the current and two previous years
        else:
            if i < 2:
                avg_ratio = country_df.loc[country_df['Year'] <= country_df.iloc[i]['Year']]\
                            ['gov_to_total_immunization_ratio'].dropna().mean()
            else:
                avg_ratio = country_df.iloc[i-2:i+1]['gov_to_total_immunization_ratio'].mean()
        # Add the average ratio to the avg_ratios_df DataFrame
        avg_ratios_df = avg_ratios_df.append({'Country': country, 'Year': country_df.iloc[i]['Year'], 
                                              'Avg_3yr_gov_ratio': avg_ratio}, ignore_index=True)

# Merge the avg_ratios_df DataFrame with the merged_df DataFrame on 'Country' and 'Year'
merged_df = pd.merge(merged_df, avg_ratios_df, on=['Country', 'Year'], how='left')


##### Create a population difference data

In [117]:
# sort the dataframe by country and year
merged_df = merged_df.sort_values(['Country', 'Year'])

In [118]:
# calculate the average coverage for different vaccines
merged_df['Avg_Vaccine_Coverage'] = merged_df[['BCG_COVERAGE', 'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE', 
                                                'HEPB_BD_COVERAGE', 'HIB3_COVERAGE', 'MCV1_COVERAGE', 'MCV2_COVERAGE',
                                                'PCV3_COVERAGE', 'POL3_COVERAGE', 'RCV1_COVERAGE', 'ROTAC_COVERAGE']].mean(axis=1)

##### Create new Coverage columns. Created label column, calculate coverage times population and land area, coverage per unit gdp, coverage change, coverage change percentage and coverage trend.

In [120]:
coverage_list = ['BCG_COVERAGE', 'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE', 'HEPB_BD_COVERAGE', 'HIB3_COVERAGE', 'MCV1_COVERAGE', 'MCV2_COVERAGE', 'PCV3_COVERAGE', 'POL3_COVERAGE', 'RCV1_COVERAGE', 'ROTAC_COVERAGE']

# convert all columns to float data type
cols_to_float = coverage_list

merged_df[cols_to_float] = merged_df[cols_to_float].astype(float)

# create the bins and labels for the coverage categories
bins = [0, 50, 75, 90, 100]
labels = ['Low', 'Medium', 'High', 'Very High']

# list of coverage columns to categorize
coverage_cols =coverage_list

# categorize each coverage column and create a new categorical column for each one
for col in coverage_cols:
    merged_df[col+'_CATEGORY'] = pd.cut(merged_df[col], bins=bins, labels=labels, include_lowest=True)


In [121]:
# Define the columns to be transformed
vaccine_cols = coverage_list

# Calculate the coverage per capita and coverage x the population
pop_list = ['total_population','total_population_0-1','total_population_1-5', 'total_population_6-10',
       'total_population_11-15', 'total_population_16-20','land_area']
for i in pop_list:
    for col in vaccine_cols:
        #merged_df[f'{col}_PER_{i}'] = merged_df[col] / merged_df[i]
        merged_df[f'{col}_TIMES_{i}'] = merged_df[col] * merged_df[i]
        #merged_df[f'{col}_PERCENTAGE'] = merged_df[col] / merged_df['total_population'] * 100

# Calculate the coverage per unit of GDP
for col in vaccine_cols:
    merged_df[f'{col}_PER_GDP'] = merged_df[col] / merged_df['GDP_Million']

# Create a new feature that captures the change in coverage over time, grouped by Country
for col in vaccine_cols:
    merged_df[f'{col}_CHANGE'] = merged_df.groupby('Country')[col].diff()

# Create a new feature that captures the change in coverage compared to the previous year as a percentage, grouped by Country
for col in vaccine_cols:
    merged_df[f'{col}_CHANGE_PERCENTAGE'] = merged_df.groupby('Country')[f'{col}_CHANGE'].pct_change() * 100

# Create a new feature that captures the overall trend in coverage over time, grouped by Country
for col in vaccine_cols:
    merged_df[f'{col}_TREND'] = merged_df.groupby('Country')[col].apply(lambda x: x.diff().fillna(0).rolling(window=3).sum()).fillna(0)


##### Created columns for previous year gdp and percentage change

In [122]:
# Define a function to create the previous year columns for GDP in million
def create_prev_year_gdp_cols_million():
    return [f"GDP_prev_{i}_year" for i in range(1, 3)]

# Create new columns for GDP, for the value for previous year up until last 5 years
prev_gdp_cols = create_prev_year_gdp_cols_million()
for i, prev_gdp_col in enumerate(prev_gdp_cols):
    merged_df[prev_gdp_col] = grouped_df['GDP_Million'].apply(lambda x: x.shift(i+1))

    # Calculate the percentage change between the current year and the previous year
    curr_col = "GDP_Million"
    pct_col = f"GDP_prev_{i+1}_year_pct_change"
    if curr_col in merged_df.columns and prev_gdp_col in merged_df.columns:
        merged_df[pct_col] = (merged_df[curr_col] - merged_df[prev_gdp_col]) / merged_df[prev_gdp_col]
    else:
        merged_df[pct_col] = float('nan')


In [123]:
# sort the dataframe by country and year
merged_df.sort_values(['Country', 'Year'], inplace=True)

# group the dataframe by country and calculate the percentage change in immunization USD between consecutive years
merged_df['current_Immunization_cost_pct_change_from_last_year'] = merged_df.groupby('Country')['Immunization USD Mil'].pct_change() * 100

##### Compute the predictive score for each column towards total immunization

In [125]:
import ppscore as pps

# Compute the PPS matrix for all pairs of columns in merged_df
pps_matrix = pps.matrix(merged_df)

# Find the columns that have a PPS score of at least 0.5 with respect to the target column
target_column = 'Immunization USD Mil'
high_pps_cols = set()
num = 1
for col in merged_df.columns:

    num = num + 1
    pps_score = pps.score(merged_df, col, target_column)['ppscore']
    if (pps_score > 0.2) & (pps_score < 0.9):
        high_pps_cols.add(col)

# Print the high-PPS columns and their corresponding PPS scores
for col in high_pps_cols:
    pps_score = pps.score(merged_df, col, target_column)['ppscore']
    print(f'{col}: {pps_score}')
    
# Create a new DataFrame with only the high-PPS columns and the target column
#new_df = merged_df[high_pps_cols + [target_column]]


In [ ]:
high_pps_cols

In [ ]:
merged_df.to_csv('immunization_prediction_data.csv')

### Data Selection

#### Selected columns to be included in prediction except Year, Country, Immunization USD Mil (target column)

In [126]:
selected_column = ['Year','Country',

 'BCG_COVERAGE_TIMES_land_area',
 'BCG_COVERAGE_TIMES_total_population_11-15',
 'BCG_COVERAGE_TIMES_total_population_16-20',
 'DTPCV1_COVERAGE_TIMES_land_area',
 'DTPCV1_COVERAGE_TIMES_total_population_16-20',
 'DTPCV3_COVERAGE_TIMES_land_area',
 'DTPCV3_COVERAGE_TIMES_total_population',
 'DTPCV3_COVERAGE_TIMES_total_population_6-10',
 'HEPB3_COVERAGE_PER_GDP',
 'HEPB3_COVERAGE_TIMES_land_area',
 'HEPB3_COVERAGE_TIMES_total_population',
 'HEPB3_COVERAGE_TIMES_total_population_6-10',
 'HEPB_BD_COVERAGE_TIMES_land_area',
 'HEPB_BD_COVERAGE_TIMES_total_population_11-15',
 'MCV1_COVERAGE_TIMES_land_area',
 'MCV2_COVERAGE_TIMES_land_area',
 'POL3_COVERAGE_TIMES_land_area',
 'RCV1_COVERAGE_TIMES_land_area',
# 'Vaccine USD Mil',
# 'gov_immunization in USD',
 'land_area',
# 'total_immunization in USD_prev_1_year_million',
# 'total_immunization in USD_prev_2_year_million',
 'total_immunization in USD_prev_3_year_avg_million',
# 'total_immunization in USD_prev_3_year_million',
 'total_population_16-20',
#  'total_vaccine in USD',
#  'total_vaccine in USD_prev_1_year_million',
#  'total_vaccine in USD_prev_2_year_million',
#  'total_vaccine in USD_prev_3_year_avg_million',
#  'total_vaccine in USD_prev_3_year_million'

 'Immunization USD Mil'
       ]

selected_data_df = merged_df[selected_column]


##### Extrapolate the data

In [127]:
x = selected_data_df.columns
cols_to_interpolate = []
for i in x:
    if i not in ['Country','Region','Gavi / Income status','Year']:
        cols_to_interpolate.append(i)

In [128]:
# Specify columns to interpolate missing values
num_rounds = 1

for round_loop in range(num_rounds):
    for j in cols_to_interpolate:
        # Group the data by country and year
        grouped = selected_data_df.groupby(['Country', 'Year'])

        # Loop through each group
        for name, group in grouped:
            # Create a new dataframe with the year as the index
            ts = pd.DataFrame(group[j].values, index=group['Year'], columns=[j])
            # Convert the column to numeric dtype
            ts[j] = pd.to_numeric(ts[j], errors='coerce')
            # Interpolate missing values using time series method
            ts = ts.interpolate(method='time')
            # Update the merged_df with the interpolated values
            selected_data_df.loc[group.index,j] = ts[j].values
            

In [129]:
selected_data_df['Year'] = selected_data_df['Year'].dt.year.astype(float)

In [130]:
selected_data_df = selected_data_df.dropna(axis = 0)

In [131]:
check_2021_data = selected_data_df[selected_data_df['Year']==2021]
check_2021_data

        Year                   Country  BCG_COVERAGE_TIMES_land_area  \
578   2021.0                    Angola                     6981520.0   
1400  2021.0                 Argentina                    22167189.0   
785   2021.0                Bangladesh                     1288683.0   
338   2021.0                   Bolivia                     8449740.0   
1401  2021.0              Burkina Faso                     2681280.0   
267   2021.0                   Burundi                      243960.0   
1402  2021.0  Central African Republic                     3800178.0   
514   2021.0                   Comoros                       17856.0   
192   2021.0             Côte d'Ivoire                     2957400.0   
123   2021.0                  Dominica                        6675.0   
1018  2021.0        Dominican Republic                      478269.0   
42    2021.0                   Ecuador                     1862700.0   
1414  2021.0                  Eswatini                      1668

In [132]:
selected_data_df.to_csv('selected_data.csv')

### ML

In [ ]:
# Import required libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np

#year to start predicting
year_pred = 2020

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Define a function to fit and predict using the best model
def fit_and_predict(best_model, X_train, X_test, y_train):
    # Train the best model on the full dataset
    best_model.fit(X_train, y_train)
    
    # Make predictions on the testing set and evaluate performance
    y_pred = best_model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return y_pred, mape, r2

# Filter data by year
train_data = selected_data_df[selected_data_df['Year'] < year_pred]
test_data = selected_data_df[selected_data_df['Year'] >= year_pred]

# Split the data into training and testing sets
target_columns = ['Immunization USD Mil']
X_col = [x for x in selected_column if x not in target_columns]
X_train = train_data[X_col].drop(['Year', 'Country'], axis=1)
X_test = test_data[X_col].drop(['Year', 'Country'], axis=1)

# Label encode categorical columns
label_encoder = LabelEncoder()
# X_train['Country'] = label_encoder.fit_transform(X_train['Country'])
# X_test['Country'] = label_encoder.transform(X_test['Country'])

# One-hot encode categorical columns (alternative to label encoding)
# ct = ColumnTransformer([('encoder', OneHotEncoder(), [0, 1, 2])], remainder='passthrough')
# X_train = ct.fit_transform(X_train)
# X_test = ct.transform(X_test)

# Define the models and their hyperparameters
models = [
    {
        'name': 'linear regression',
        'model': LinearRegression(),
        'params': {}
    },
    {
        'name': 'random forest',
        'model': RandomForestRegressor(),
        'params': {
            'model__n_estimators': [10, 50, 100, 200],
            'model__max_depth': [None, 5, 10, 20]
        }
    },
    {
        'name': 'gradient boosting',
        'model': GradientBoostingRegressor(),
        'params': {
            'model__n_estimators': [10, 50, 100, 200],
            'model__max_depth': [3, 5, 10],
            'model__learning_rate': [0.1, 0.01, 0.001]
        }
    },
#     {
#         'name': 'neural network',
#         'model': MLPRegressor(activation='relu'),
#         'params': {
#             'model__hidden_layer_sizes': [(50,), (100,), (50, 50)],
#             'model__learning_rate_init': [0.1, 0.01, 0.001],
#             'model__max_iter': [500],
#             'model__solver': ['adam'],
#         }
#     }
]

# Evaluate each model using cross-validation and select the best one for each target variable
best_models = {}
for target_column in target_columns:
    y_train = train_data[target_column]
    y_test = test_data[target_column]
    best_model = None
    best_score = None
    for model in models:
        pipeline = Pipeline([('scaler', StandardScaler()), ('model', model['model'])])
        clf = GridSearchCV(pipeline, model['params'], cv=10, scoring='neg_mean_absolute_percentage_error')
    
        clf.fit(X_train, y_train)
        score = -clf.best_score_
        print(model['name'], f'mean absolute percentage error ({target_column}):', score)
        if best_score is None or score < best_score:
            best_score = score
            best_model = clf.best_estimator_
    # Save the best model for the current target variable
    best_models[target_column] = best_model

#Get the name and parameters of the best model for each target variable
best_model_names = {target_column: best_models[target_column].named_steps['model'].__class__.__name__ for target_column in target_columns}
best_model_params = {target_column: best_models[target_column].named_steps['model'].get_params() for target_column in target_columns}
print('Best models:', best_model_names)
print('Best model parameters:', best_model_params)

for target_column in target_columns:
    best_model = best_models[target_column]
    y_pred, mape, r2 = fit_and_predict(best_model, X_train, X_test, train_data[target_column])

    # Append the predictions and error to the DataFrame
    selected_data_df[f'{target_column}_predicted'] = np.nan
    selected_data_df.loc[selected_data_df['Year'] >= year_pred, f'{target_column}_predicted'] = y_pred

print(f'mean absolute percentage error ({target_columns[0]}) {mean_absolute_percentage_error(y_test, y_pred)}%')
selected_data_df['mape'] = selected_data_df.apply(lambda x: mean_absolute_percentage_error(x['Immunization USD Mil'], x['Immunization USD Mil_predicted']), axis=1)


linear regression mean absolute percentage error (Immunization USD Mil): 1.7211682445001881
random forest mean absolute percentage error (Immunization USD Mil): 0.6037041302211532


In [ ]:
print(f'mean absolute percentage error ({target_columns[0]}) {mean_absolute_percentage_error(y_test, y_pred)}%')
# Calculate and print the R2 score
r2 = r2_score(y_test, y_pred)
print(f'R2 score: {r2}')

##### Reinflate the data and output

In [ ]:
selected_data_df['prediction'] = ['underprediction' if (x['Immunization USD Mil_predicted'] - x['Immunization USD Mil']) < 0 else 'overprediction' for idx, x in selected_data_df.iterrows()]
predicted_data = selected_data_df[selected_data_df['Year'] >= year_pred]

cols_to_adjust = ['Vaccine Delivery Cost USD Mil','Immunization USD Mil_predicted','Immunization USD Mil','Vaccine USD Mil','total_vaccine in USD', 'total_immunization in USD', 'gov_immunization in USD', 'total_immunization in USD per surviving infant', 'total_vaccine in USD per surviving infant', 'GDP_Million']
origin_merged_df['Year']= pd.to_datetime(origin_merged_df['Year'], format='%Y').dt.strftime('%Y')
predicted_data['Year']= pd.to_datetime(predicted_data['Year'], format='%Y').dt.strftime('%Y')

    
output = pd.merge(predicted_data[['Country', 'Year','Immunization USD Mil_predicted','mape','prediction']], origin_merged_df, on = ['Country','Year'], how = 'inner')
for col in cols_to_adjust:
    output[f'{col}'] = (output[col] * output['gdp deflator index']) / 100
output.to_csv('cleaned_data_with_predictions.csv', index=False)